In [1]:
import os
import numpy as np
import PIL
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import keras
from keras.models import Model, Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.applications.vgg19 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop
from keras import activations
from keras.preprocessing import image

from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

Using TensorFlow backend.


In [2]:
def path_join(dirname, filenames):
    return [os.path.join(dirname, filename) for filename in filenames]

In [3]:
# Data augmentation and pre-processing

datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=[0.9, 1.5],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

datagen_test = ImageDataGenerator(rescale=1./255)

In [4]:
# Directory setup

train_dir = '../train_data/'
test_dir = '../train_test_data/'

input_shape = (224, 224)
batch_size = 32

In [5]:
# Making data generator
generator_train = datagen_train.flow_from_directory(
    directory=train_dir,
    target_size=input_shape,
    batch_size=batch_size,
    shuffle=True
)

generator_test = datagen_test.flow_from_directory(
    directory=test_dir,
    target_size=input_shape,
    batch_size=batch_size,
    shuffle=False
)

Found 5629 images belonging to 5 classes.
Found 623 images belonging to 5 classes.


In [6]:
# Setting epoch size for batch step calculation
epochs = 50

# Steps per epoch setup
steps_per_epoch = generator_train.n / batch_size
steps_test = generator_test.n / batch_size

In [7]:
# joing directory and filenames
image_paths_train = path_join(train_dir, generator_train.filenames)
image_paths_test = path_join(test_dir, generator_test.filenames)

# classes available
cls_train = generator_train.classes
cls_test = generator_test.classes

class_names = list(generator_train.class_indices.keys())
num_classes = generator_train.num_classes

In [8]:
# In case of unequal number of dataset, use class weight balancing
class_weight = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(cls_train),
    y=cls_train
)

### Create model

In [9]:
vgg_conv = keras.applications.vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

model = Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 5125      
Total params: 45,720,645
Trainable params: 32,775,685
Non-trainable params: 12,944,960
_________________________________________________________________


In [10]:
# setting learning rate
optimizer = Adam(lr=1e-5)

# Loss definition
loss = 'categorical_crossentropy'

# Metrics for measurement
metrics = ['categorical_accuracy']

In [11]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4694631928490248610
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11272401716
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11447008020918531338
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [12]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [13]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Begin training for transfer learning
history = model.fit_generator(
    generator=generator_train,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    class_weight=class_weight,
    validation_data=generator_test,
    validation_steps=steps_test
)

Epoch 1/50
176/175 [==============================] - 90s 511ms/step - loss: 1.1244 - categorical_accuracy: 0.5428 - val_loss: 0.7626 - val_categorical_accuracy: 0.7047
Epoch 2/50
176/175 [==============================] - 80s 452ms/step - loss: 0.7208 - categorical_accuracy: 0.7173 - val_loss: 0.6879 - val_categorical_accuracy: 0.7368
Epoch 3/50
176/175 [==============================] - 81s 461ms/step - loss: 0.6244 - categorical_accuracy: 0.7475 - val_loss: 0.6502 - val_categorical_accuracy: 0.7657
Epoch 4/50
176/175 [==============================] - 80s 457ms/step - loss: 0.5727 - categorical_accuracy: 0.7763 - val_loss: 0.5854 - val_categorical_accuracy: 0.7849
Epoch 5/50
176/175 [==============================] - 82s 467ms/step - loss: 0.5226 - categorical_accuracy: 0.8007 - val_loss: 0.5308 - val_categorical_accuracy: 0.8074
Epoch 6/50
176/175 [==============================] - 81s 459ms/step - loss: 0.4874 - categorical_accuracy: 0.8135 - val_loss: 0.5361 - val_categorical_acc

176/175 [==============================] - 81s 459ms/step - loss: 0.1772 - categorical_accuracy: 0.9294 - val_loss: 0.5288 - val_categorical_accuracy: 0.8427
Epoch 50/50
176/175 [==============================] - 83s 472ms/step - loss: 0.1798 - categorical_accuracy: 0.9325 - val_loss: 0.4543 - val_categorical_accuracy: 0.8636


In [14]:
model.save('k2.h5')

### Check Performance

In [15]:
validation_generator = datagen_test.flow_from_directory(
    directory=test_dir,
    target_size=input_shape,
    batch_size=batch_size,
    shuffle=False
)

# Get the filenames from the generator
fnames = validation_generator.filenames
 
# Get the ground truth from generator
ground_truth = validation_generator.classes
 
# Get the label to class mapping from the generator
label2index = validation_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model.predict_generator(
    validation_generator,
    steps=validation_generator.samples/validation_generator.batch_size,
    verbose=1
)
predicted_classes = np.argmax(predictions, axis=1)

Found 623 images belonging to 5 classes.
20/19 [==============================] - 7s 340ms/step


In [17]:
from sklearn.metrics import f1_score
f1 = f1_score(ground_truth, predicted_classes, average='weighted')
print(f1)

0.8605983104776662
